# Self Catalysis

1. Find alpha =/> beta and beta =/> alpha pairs
2. Calculate alpha x alpha (alpha') and beta x alpha(beta')
3. Sort alpha' and beta'
4. Check if alpha' -> beta'

In [ ]:
from keras.models import model_from_json

## Data preprocessing

In [ ]:
%run ../ml_models/CommonFunctions.ipynb

In [ ]:
# dim = 4
# optimizer = 'adam'

In [ ]:
df_sample = read_csv('../data/output_d'+str(dim)+'_s1000000.csv')
df_input = format_input(df_sample)

## Filtering not majorated samples

In [ ]:
df_not_majorated = df_input[df_input['AB'] == False]
#TODO filter only b =/> a to get stronger results

In [ ]:
df_not_majorated.head()

## Multiplying and sorting elements

In [ ]:
def multiply_and_sort(row):
    newA = np.sort(np.multiply.outer(row["A"], row["A"]).flatten())[::-1]
    newB = np.sort(np.multiply.outer(row["B"], row["A"]).flatten())[::-1]
    return [newA, newB, is_a_valid_pair(newA,newB)]

df_self_catalysis = df_not_majorated.reset_index(drop=True).apply(multiply_and_sort, axis=1, result_type='broadcast')

## Loading model and weights

In [ ]:
model_filename = './results/model/seq_200relu_100relu_50ep__d'+str(dim**2)+'_opt-'+optimizer
json_file = open(model_filename+'.json')
loaded_model_json = json_file.read()
json_file.close()

loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights(model_filename+'.h5')
loaded_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
score = loaded_model.evaluate([np.array(list(df_self_catalysis['A'])), np.array(list(df_self_catalysis['B']))], np.array(list(df_self_catalysis['AB'])))

In [ ]:
score

In [ ]:
output_name = 'results/self_catalysis_acc.csv'
save_data(output_name,optimizer,dim,score[1])